In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [19]:
!pip install dask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import dask.dataframe as dd
import numpy as np 
import pandas as pd
from dask import delayed
from tqdm.auto import tqdm
tqdm.pandas(desc ="processing_rows")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
hcp = pd.read_excel("/content/drive/MyDrive/IITK Hackathon/kdsh-data-solution-task-1/HCP_Data_KDAG_Hackathon/HCP_Data_KDAG_Hackathon.xlsx")
patient =pd.read_excel("/content/drive/MyDrive/IITK Hackathon/kdsh-data-solution-task-1/Patient_Data_KDAG_Hackathon/Patient_Data_KDAG_Hackathon.xlsx")

In [8]:
patient.head()

,Patient ID,Physician_ID,Date_of_Visit,Year of Birth,Gender,Geographical_State
0,16445,axt01951,2020-01-12,1962.0,F,ME
1,16445,axt01951,2019-09-25,1962.0,F,ME
2,16445,axt01951,2019-02-12,1962.0,F,ME
3,16445,axt01951,2019-11-07,1962.0,F,ME
4,16445,axt01951,2019-04-17,1962.0,F,ME


In [9]:
patient.nunique()

Patient ID            39956
Physician_ID           9229
Date_of_Visit           393
Year of Birth            65
Gender                    3
Geographical_State       57
dtype: int64

In [10]:
hcp.head()

,Physician_ID,Time_Period,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Physician_Segment,Emails_Delivered,Speaker_Programs_Attended,Vouchers_Dropped,Specialty
0,axt00001,20190104,0.0,2.80567,1,0,3-Low,0,0,0,Dermatologist
1,axt00001,20190111,0.0,20.57312,1,0,3-Low,0,0,0,Dermatologist
2,axt00001,20190118,0.0,6.16010,1,0,3-Low,0,0,0,Dermatologist
3,axt00001,20190125,0.0,8.95501,1,5,3-Low,0,0,0,Dermatologist
4,axt00001,20190201,0.0,9.13793,1,0,3-Low,0,0,0,Dermatologist


In [11]:
hcp.nunique()

Physician_ID                 10000
Time_Period                     57
Brand_Rx                      8699
Market_Rx                    20766
Sales_Rep_Calls                  2
Samples_Dropped                 81
Physician_Segment                3
Emails_Delivered                 9
Speaker_Programs_Attended        3
Vouchers_Dropped                20
Specialty                        3
dtype: int64

In [12]:
print(hcp.Vouchers_Dropped.unique())

[ 0  2  5  4  1  8 20  6 10  9  3  7 15 14 13 12 18 16 11 23]


In [13]:
pd.set_option('display.max_rows' ,None)
hcp.head(20)

,Physician_ID,Time_Period,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Physician_Segment,Emails_Delivered,Speaker_Programs_Attended,Vouchers_Dropped,Specialty
0,axt00001,20190104,0.000,2.80567,1,0,3-Low,0,0,0,Dermatologist
1,axt00001,20190111,0.000,20.57312,1,0,3-Low,0,0,0,Dermatologist
2,axt00001,20190118,0.000,6.16010,1,0,3-Low,0,0,0,Dermatologist
3,axt00001,20190125,0.000,8.95501,1,5,3-Low,0,0,0,Dermatologist
4,axt00001,20190201,0.000,9.13793,1,0,3-Low,0,0,0,Dermatologist
5,axt00001,20190208,0.841,10.99941,1,9,3-Low,1,0,0,Dermatologist
6,axt00001,20190215,0.000,2.31340,1,0,3-Low,1,0,0,Dermatologist
7,axt00001,20190222,0.000,11.97588,0,0,3-Low,0,0,0,Dermatologist
8,axt00001,20190301,0.000,17.52804,0,0,3-Low,0,0,0,Dermatologist
9,axt00001,20190308,0.000,9.15138,0,0,3-Low,1,0,0,Dermatologist


In [14]:
hcp['Physician_Segment'].unique()

array(['3-Low', '1-High', '2-Medium'], dtype=object)

In [15]:
hcp.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Time_Period,570000.0,2.019150e+07,2693.161915,20190104.0,20190412.0,2.019072e+07,2.019102e+07,2.020013e+07
Brand_Rx,570000.0,6.828282e-01,1.369037,0.0,0.0,0.000000e+00,1.004000e+00,6.425500e+01
Market_Rx,570000.0,7.988949e+00,10.599537,0.0,0.0,5.081410e+00,1.091064e+01,2.599670e+02
Sales_Rep_Calls,570000.0,3.912947e-01,0.488041,0.0,0.0,0.000000e+00,1.000000e+00,1.000000e+00
Samples_Dropped,570000.0,8.657193e-01,3.437103,0.0,0.0,0.000000e+00,0.000000e+00,1.560000e+02
Emails_Delivered,570000.0,2.711737e-01,0.710370,0.0,0.0,0.000000e+00,0.000000e+00,8.000000e+00
Speaker_Programs_Attended,570000.0,5.745614e-03,0.076183,0.0,0.0,0.000000e+00,0.000000e+00,2.000000e+00
Vouchers_Dropped,570000.0,4.204386e-02,0.448897,0.0,0.0,0.000000e+00,0.000000e+00,2.300000e+01


In [16]:
hcp.head(6)

,Physician_ID,Time_Period,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Physician_Segment,Emails_Delivered,Speaker_Programs_Attended,Vouchers_Dropped,Specialty
0,axt00001,20190104,0.000,2.80567,1,0,3-Low,0,0,0,Dermatologist
1,axt00001,20190111,0.000,20.57312,1,0,3-Low,0,0,0,Dermatologist
2,axt00001,20190118,0.000,6.16010,1,0,3-Low,0,0,0,Dermatologist
3,axt00001,20190125,0.000,8.95501,1,5,3-Low,0,0,0,Dermatologist
4,axt00001,20190201,0.000,9.13793,1,0,3-Low,0,0,0,Dermatologist
5,axt00001,20190208,0.841,10.99941,1,9,3-Low,1,0,0,Dermatologist


In [17]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 874 kB/s 


In [18]:
from category_encoders import OrdinalEncoder 

In [20]:
# maplist = [{'col': 'Physician_Segment', 'mapping': {'3-Low': 1, '2-Medium': 2,'1-High': 3}}]
# oe = OrdinalEncoder(mapping=maplist)

d = {'3-Low': 1, '2-Medium': 2,'1-High': 3}
hcp['Physician_Segment'] = hcp['Physician_Segment'].map(d)

In [21]:
hcp.head(30)

,Physician_ID,Time_Period,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Physician_Segment,Emails_Delivered,Speaker_Programs_Attended,Vouchers_Dropped,Specialty
0,axt00001,20190104,0.000,2.80567,1,0,1,0,0,0,Dermatologist
1,axt00001,20190111,0.000,20.57312,1,0,1,0,0,0,Dermatologist
2,axt00001,20190118,0.000,6.16010,1,0,1,0,0,0,Dermatologist
3,axt00001,20190125,0.000,8.95501,1,5,1,0,0,0,Dermatologist
4,axt00001,20190201,0.000,9.13793,1,0,1,0,0,0,Dermatologist
5,axt00001,20190208,0.841,10.99941,1,9,1,1,0,0,Dermatologist
6,axt00001,20190215,0.000,2.31340,1,0,1,1,0,0,Dermatologist
7,axt00001,20190222,0.000,11.97588,0,0,1,0,0,0,Dermatologist
8,axt00001,20190301,0.000,17.52804,0,0,1,0,0,0,Dermatologist
9,axt00001,20190308,0.000,9.15138,0,0,1,1,0,0,Dermatologist


In [22]:
useful_columns=['Physician_ID','Time_Period','Brand_Rx','Market_Rx','Sales_Rep_Calls','Samples_Dropped','Physician_Segment','Emails_Delivered','Speaker_Programs_Attended','Vouchers_Dropped']

In [23]:
df=hcp[useful_columns]
df.head()

,Physician_ID,Time_Period,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Physician_Segment,Emails_Delivered,Speaker_Programs_Attended,Vouchers_Dropped
0,axt00001,20190104,0.0,2.80567,1,0,1,0,0,0
1,axt00001,20190111,0.0,20.57312,1,0,1,0,0,0
2,axt00001,20190118,0.0,6.16010,1,0,1,0,0,0
3,axt00001,20190125,0.0,8.95501,1,5,1,0,0,0
4,axt00001,20190201,0.0,9.13793,1,0,1,0,0,0


In [24]:

useful_columns=['Physician_ID','Time_Period','Market_Rx','Sales_Rep_Calls','Samples_Dropped','Physician_Segment','Emails_Delivered','Speaker_Programs_Attended','Vouchers_Dropped']

In [25]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[useful_columns], df[['Brand_Rx']], test_size=0.25, random_state=42,stratify=df[['Physician_ID']])

In [26]:
X_train.head()
try:
    X_train=X_train.drop(columns=['Physician_ID'])
except:
    pass

try:
    X_test=X_test.drop(columns=['Physician_ID'])
except:
    pass

ANN 

In [27]:
from sklearn.preprocessing import StandardScaler
scalerX = StandardScaler()
print(scalerX.fit(X_train))
scalerY = StandardScaler()
print(scalerY.fit(y_train))

StandardScaler()
StandardScaler()


In [28]:
X_tra=scalerX.transform(X_train)
X_tes=scalerX.transform(X_test)
y_tra=scalerY.transform(y_train)
y_tes=scalerY.transform(y_test)

In [29]:
input_shape=[X_tra.shape[1]]

In [30]:
import tensorflow as tf
model = tf.keras.Sequential([
 
    tf.keras.layers.Dense(units=64, activation='relu',
                          input_shape=input_shape),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1)
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,801
Trainable params: 4,801
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(optimizer='adam',loss=tf.keras.losses.MeanSquaredError()) 

In [32]:
losses = model.fit(X_tra, y_tra,
                   validation_data=(X_tes, y_tes),
                   batch_size=16,
                   epochs=25,  # total epoch
 
                   )

Epoch 1/25
26719/26719 [==============================] - 86s 3ms/step - loss: 0.5291 - val_loss: 0.5357
Epoch 2/25
26719/26719 [==============================] - 78s 3ms/step - loss: 0.5213 - val_loss: 0.5241
Epoch 3/25
26719/26719 [==============================] - 83s 3ms/step - loss: 0.5198 - val_loss: 0.5290
Epoch 4/25
26719/26719 [==============================] - 76s 3ms/step - loss: 0.5171 - val_loss: 0.5295
Epoch 5/25
26719/26719 [==============================] - 85s 3ms/step - loss: 0.5170 - val_loss: 0.5253
Epoch 6/25
26719/26719 [==============================] - 82s 3ms/step - loss: 0.5152 - val_loss: 0.5234
Epoch 7/25
26719/26719 [==============================] - 83s 3ms/step - loss: 0.5156 - val_loss: 0.5226
Epoch 8/25
26719/26719 [==============================] - 75s 3ms/step - loss: 0.5150 - val_loss: 0.5216
Epoch 9/25
26719/26719 [==============================] - 82s 3ms/step - loss: 0.5143 - val_loss: 0.5199
Epoch 10/25
26719/26719 [==============================

In [33]:
model.evaluate(X_tes, y_tes)

4454/4454 [==============================] - 8s 2ms/step - loss: 0.5161


0.5160594582557678

In [34]:
print(model.metrics_names)

['loss']


In [35]:
res=model.predict(X_tes)

4454/4454 [==============================] - 6s 1ms/step


In [36]:
sales_columns=['Sales_Rep_Calls','Samples_Dropped','Emails_Delivered','Physician_Segment','Speaker_Programs_Attended','Vouchers_Dropped']

In [37]:
sales_call_max=hcp.Sales_Rep_Calls.unique().max()
samples_max=hcp.Samples_Dropped.unique().max()
emails_max=hcp.Emails_Delivered.unique().max()
dict_maxvals={0:sales_call_max,1:samples_max,2:emails_max}

In [38]:
mse=tf.keras.losses.MeanSquaredError()

In [39]:
def get_best_channel(phy_id):
    gg=hcp.loc[hcp['Physician_ID'] == phy_id]
    X=gg[useful_columns]
    y=gg['Brand_Rx'].values
    gg=X.drop(columns=['Physician_ID'])
    dict_max={0:[0,0],1:[0,0],2:[0,0]}
    for i in range(3):
        max_val=[0,1000000]
        for j in range(dict_maxvals[i]+1):
            for col in sales_columns:
                gg[col] = gg[col].replace([hcp[col].unique()],0)
#             print("Column is ",sales_columns[i]," Frequency = ",j)
            gg[sales_columns[i]] = gg[sales_columns[i]].replace([hcp[sales_columns[i]].unique()],j)
            X=scalerX.transform(gg)
            res=model.predict(X)
            y_pred=scalerY.inverse_transform(res)
            err=(mse(y,y_pred).numpy().sum())/X.shape[0]
            if (err<=max_val[1]):
#                 print("*************** New Min Erro = *************** ",err)
                max_val=[j,err]
        dict_max[i]=max_val   
    res=[1000,0,0]
    for i in range(3):
        if(res[0]>=dict_max[i][1]):
            res=[dict_max[i][1],i,dict_max[i][0]]
    return res
    
    

In [40]:
print(get_best_channel('axt00001'))

2/2 [==============================] - 0s 3ms/step
[0.018976749035350064, 2, 3]


In [41]:
sub=pd.read_csv('/content/drive/MyDrive/IITK Hackathon/kdsh-data-solution-task-1/sample_submission.csv')
sub.head()

,Physician_ID,Channel
0,axt00001,1
1,axt00002,1
2,axt00003,1
3,axt00004,1
4,axt00005,1


In [42]:
for i in range(sub.shape[0]):
    cl=sub.Physician_ID.iloc[i]
    res=get_best_channel(cl)
    sub.loc[i,['Channel']]=res[1]

2/2 [==============================] - 0s 8ms/step


KeyboardInterrupt: ignored